In [1]:
import numpy as np

In [2]:
# read in the text, transforming everything to lower case
text_org = open('Beiras.txt', encoding="utf-8").read().lower()
print('our original text has ' + str(len(text_org)) + ' characters')

our original text has 1213561 characters


In [3]:
import re
text_without_source="";
regexp=re.compile(r'http')
for line in text_org.splitlines():
    if not regexp.search(line):
        text_without_source= text_without_source + line

In [4]:
chars=sorted(list(set(text_without_source)))
print(chars);

[' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '¨', 'ª', '«', '\xad', '´', 'º', '»', '¿', 'à', 'á', 'â', 'ã', 'ä', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', 'ú', 'ü', '–', '—', '‘', '’', '“', '”', '•', '…']


In [5]:

text_clean = re.sub('[ºªàâäçèêïìôöü&%@•…«»”“*/!"(),.:;_¿¡¿‘’´\[\]\']',' ',text_without_source)
text_clean = text_clean.replace("  "," ")
chars=sorted(list(set(text_clean )))
print(chars);
print ("this corpus has " +  str(len(text_clean)) + " total number of characters")
print ("this corpus has " +  str(len(chars)) + " unique characters")


[' ', '$', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¨', '\xad', 'á', 'ã', 'é', 'ë', 'í', 'î', 'ñ', 'ò', 'ó', 'õ', 'ú', '–', '—']
this corpus has 1174380 total number of characters
this corpus has 55 unique characters


In [6]:
def window_transform_text(text,window_size,step_size):
    # containers for input/output pairs
    inputs = []
    outputs = []
    #Number of windows to create
    n_windows=int((len(text) - window_size)/ step_size)
    for j in range(n_windows) :
        # k .- Start index
        k= j * step_size
        inputs.append(text[k:(k+window_size)])
        outputs.append(text[k+window_size])

    return inputs,outputs

In [7]:
# run your text window-ing function 
window_size = 100
step_size = 1

In [8]:
# this dictionary is a function mapping each unique character to a unique integer
chars_to_indices = dict((c, i) for i, c in enumerate(chars))  # map each unique character to unique integer

# this dictionary is a function mapping each unique integer back to a unique character
indices_to_chars = dict((i, c) for i, c in enumerate(chars))  # map each unique integer back to unique character

In [9]:
# transform character-based input/output into equivalent numerical versions
def encode_io_pairs(text,window_size,step_size):
    # number of unique chars
    chars = sorted(list(set(text)))
    num_chars = len(chars)
    
    # cut up text into character input/output pairs
    inputs, outputs = window_transform_text(text,window_size,step_size)
    
    # create empty vessels for one-hot encoded input/output
    X = np.zeros((len(inputs), window_size, num_chars), dtype=np.bool)
    y = np.zeros((len(inputs), num_chars), dtype=np.bool)
    
    # loop over inputs/outputs and tranform and store in X/y
    for i, sentence in enumerate(inputs):
        for t, char in enumerate(sentence):
            X[i, t, chars_to_indices[char]] = 1
        y[i, chars_to_indices[outputs[i]]] = 1
        
    return X,y

In [10]:
# use your function
X,y = encode_io_pairs(text_clean,window_size,step_size)

In [11]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2359881874260666339, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 3879469056
 locality {
   bus_id: 1
 }
 incarnation: 17420852638305713478
 physical_device_desc: "device: 0, name: GRID K520, pci bus id: 0000:00:03.0"]

In [12]:
def add_log(log_string):
    with open("log-hyper.txt", "a") as myfile:
        myfile.write(log_string + "\n")  
        

In [13]:
# function that uses trained model to predict a desired number of future characters
def predict_next_chars(model,input_chars,num_to_predict):     
    # create output
    predicted_chars = ''
    for i in range(num_to_predict):
        # convert this round's predicted characters to numerical input    
        x_test = np.zeros((1, window_size, len(chars)))
        for t, char in enumerate(input_chars):
            x_test[0, t, chars_to_indices[char]] = 1.

        # make this round's prediction
        test_predict = model.predict(x_test,verbose = 0)[0]

        # translate numerical prediction back to characters
        r = np.argmax(test_predict)                           # predict class of each test input
        d = indices_to_chars[r] 

        # update predicted_chars and input
        predicted_chars+=d
        input_chars+=d
        input_chars = input_chars[1:]
    return predicted_chars

def print_predicctions(model,weights_file):
    start_inds = [100,1000,5000]

    # load in weights
    model.load_weights(weights_file)
    for s in start_inds:
        start_index = s
        input_chars = text_clean[start_index: start_index + window_size]

        # use the prediction function
        predict_input = predict_next_chars(model,input_chars,num_to_predict = 100)

        # print out input characters
        print('------------------')
        input_line = 'input chars = ' + '\n' +  input_chars + '"' + '\n'
        print(input_line)

        # print out predicted characters
        line = 'predicted chars = ' + '\n' +  predict_input + '"' + '\n'
        print(line)  
        add_log(input_chars + "..." + predict_input)

In [14]:
### necessary functions from the keras library
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import keras
import random
from keras.layers import Dense, Activation,GRU
def create_gru_model(chars):
    num_chars = len(chars)
    model= Sequential()
    # 1 Layer .- LSTM layer 1 should be an LSTM module with 200 hidden units
    model.add(GRU(200,input_shape = (window_size,num_chars),return_sequences=True))
    # 2 Layer .-  Dense, with number chars unit and softmax activation
    model.add(GRU(200))
    model.add(Dense(num_chars,activation='softmax'))
    # initialize optimizer
    optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
    # compile model --> make sure initialized optimizer and callbacks - as defined above - are used
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    return model

Using TensorFlow backend.


loss='categorical_crossentropy'
RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

sobre 1225 s train

loss='categorical_crossentropy'
RMSprop(lr=0.01, rho=0.9, epsilon=1e-08, decay=0.0)

In [15]:
epoch=30

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_epoch_end(self, batch, logs={}):
        add_log(str(logs))
history = LossHistory()
import timeit

In [ ]:
add_log("\n\nlr 0.001 RMSprot bacch 500")
model=create_gru_model(chars)
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=500, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('model_weights/best_beiras_gru_textdata_weights-lr001.hdf5')
print_predicctions(model,'model_weights/best_beiras_gru_textdata_weights-lr01.hdf5')


/home/aind2/anaconda3/envs/beiras-rnn/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/30
 547000/1174280 [============>.................] - ETA: 645s - loss: 2.0926 

In [ ]:
add_log("\n\nlr 0.01 RMSprot bacch 500")
model=create_gru_model(chars)
optimizer = keras.optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=1e-08, decay=0.0)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=500, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('model_weights/best_beiras_gru_textdata_weights-lr01.hdf5')
print_predicctions(model,'model_weights/best_beiras_gru_textdata_weights-lr01.hdf5')



In [ ]:
add_log("\n\nlr 0.1 RMSprot bacch 500")
model=create_gru_model(chars)
optimizer = keras.optimizers.RMSprop(lr=0.1, rho=0.9, epsilon=1e-08, decay=0.0)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=500, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('model_weights/best_beiras_gru_textdata_weights-lr1.hdf5')
print_predicctions(model,'model_weights/best_beiras_gru_textdata_weights-lr1.hdf5')

In [ ]:
add_log("\n\nlr 0.001 Adam bacch 500")
model=create_gru_model(chars)
optimizer = keras.optimizers.Adam(lr=0.001)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=500, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('model_weights/best_beiras_gru_textdata_weights-adam-lr001.hdf5')
print_predicctions(model,'model_weights/best_beiras_gru_textdata_weights-adam-lr001.hdf5')

In [ ]:
add_log("\n\nlr 0.01 Adam bacch 500")
model=create_gru_model(chars)
optimizer = keras.optimizers.Adam(lr=0.01)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=500, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('model_weights/best_beiras_gru_textdata_weights-adam-lr01.hdf5')
print_predicctions(model,'model_weights/best_beiras_gru_textdata_weights-adam-lr01.hdf5')

In [ ]:
add_log("\n\nlr 0.1 Adam bacch 500")
model=create_gru_model(chars)
optimizer = keras.optimizers.Adam(lr=0.1)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=500, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('model_weights/best_beiras_gru_textdata_weights-adam-lr1.hdf5')
print_predicctions(model,'model_weights/best_beiras_gru_textdata_weights-adam-lr1.hdf5')

In [ ]:
add_log("\n\nlr 0.001 RSProp bacch 32")
model=create_gru_model(chars)
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=32, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('model_weights/best_beiras_gru_textdata_weights-batch32.hdf5')
print_predicctions(model,'model_weights/best_beiras_gru_textdata_weights-batch32.hdf5')

In [ ]:
add_log("\n\nlr 0.001 RSProp bacch 5000")
model=create_gru_model(chars)
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=5000, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('model_weights/best_beiras_gru_textdata_weights-batch5000.hdf5')
print_predicctions(model,'model_weights/best_beiras_gru_textdata_weights-batch5000.hdf5')